In [1]:
from google.colab import drive

In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import pandas as pd

1. Take a look at the three datasets (abbrevs, pop, areas: 'state-abbrevs_my.csv‘,
'state-areas_my.csv‘, 'state-pop_my.csv'), using the Pandas read_csv() function
and head()

In [4]:
df_area = pd.read_csv("/content/drive/MyDrive/concat3/areas.csv", delimiter=";")
df_area.head(10)

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954
8,Florida,65758
9,Georgia,59441


In [5]:
df_abbr = pd.read_csv("/content/drive/MyDrive/concat3/abbr.csv", delimiter=";")
df_abbr.head(10)

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,District of Columbia,DC
9,Florida,FL


In [6]:
df_pop = pd.read_csv("/content/drive/MyDrive/concat3/pop.csv", delimiter=";")
df_pop.head(10)

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0
5,AL,total,2011,4801627.0
6,AL,total,2009,4757938.0
7,AL,under18,2009,1134192.0
8,AL,under18,2013,1111481.0
9,AL,total,2013,4833722.0


2. Start with a many-to-one merge (name new dataframe merged) that will give
us the full state name within the population DataFrame (use how=? to make
sure no data is thrown away due to mismatched labels). Check result by using
head(). Drop redundant column ‘abbreviation’.

In [7]:
df_result1 = pd.merge(df_pop, df_abbr.set_index("abbreviation"), right_index=True, left_on="state/region", how="outer")
df_result1

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


3. Let’s double-check whether there were any mismatches here, which we
can do by looking for rows with nulls (by using isnull().any() method)

In [8]:
df_result1.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

4. Some of the ........(column_name) info is null; let’s figure out which these are!
(select column needed for it, use isnull(), output with head() – in 1 cell)

In [9]:
df_result1[df_result1.state.isnull() | df_result1.population.isnull()].head(20)

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


5. We see also that some of the new state entries are also null, which means
that there was no corresponding entry in the abbrevs key! Let’s figure
out which regions lack this match (by using loc indexer select column needed for
checking, use isnull(), specify column needed for output, use unique() method – in
1 cell)

In [10]:
df_result1[df_result1.state.isnull()].loc(1)["state/region"].unique()

array(['PR', 'USA'], dtype=object)

6. Our population data (pop) includes 2 entries (result of prev.step) for .....
\state1_name\ (....\abbr from ‘state/region’ column \) and ......
\state2_name\ (....), while these entries do not appear in the state abbreviation
key. We should fix these quickly by filling in appropriate entries (use loc indexer
to select and assign new values)

In [11]:
df_filled = df_result1.copy()
for i in df_filled[df_filled.state.isnull()].index:
  df_filled.loc[i, "state"] = "Puerto Rico" if df_filled.loc[i, "state/region"] == "PR" else "United States"

df_filled.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

7. No more nulls in the ‘state’ column: we’re all set. Now you can
merge the result (final dataset) with the area data.

In [12]:
df_final = pd.merge(df_filled, df_area, on="state", how="outer")
df_final.head(10)

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
5,AL,total,2011,4801627.0,Alabama,52423.0
6,AL,total,2009,4757938.0,Alabama,52423.0
7,AL,under18,2009,1134192.0,Alabama,52423.0
8,AL,under18,2013,1111481.0,Alabama,52423.0
9,AL,total,2013,4833722.0,Alabama,52423.0


8. Again, let’s check for nulls to see if there were any mismatches

In [13]:
df_final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [14]:
df_final.to_csv("/content/drive/MyDrive/concat3/final.csv")